In [5]:
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
from torchvision.transforms import Compose, ToTensor, Normalize
from torchvision.transforms.functional import resize

# Load MiDaS model
midas = torch.hub.load("intel-isl/MiDaS", "MiDaS_small")  # Use "DPT_Large" for better quality
midas.eval()

# Move to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
midas.to(device)

# Define preprocessing pipeline
transform = Compose([
    ToTensor(),
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


Using cache found in C:\Users\OWNER/.cache\torch\hub\intel-isl_MiDaS_master
c:\Users\OWNER\anaconda3\envs\gpt_env\lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


Loading weights:  None


c:\Users\OWNER\anaconda3\envs\gpt_env\lib\site-packages\torch\hub.py:330: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/rwightman/gen-efficientnet-pytorch/zipball/master" to C:\Users\OWNER/.cache\torch\hub\master.zip
Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/tf_efficientnet_lite3-b733e338.pth" to C:\Users\OWNER/.cache\torch\hub\checkpoints\tf_efficientnet_lite3-b733e338

In [6]:
def estimate_depth(image_path):
    # Load and preprocess the image
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_input = transform(img).unsqueeze(0).to(device)

    # Predict depth
    with torch.no_grad():
        depth_map = midas(img_input)
    
    # Convert depth map to NumPy array
    depth_map = depth_map.squeeze().cpu().numpy()

    # Normalize for visualization
    depth_map = (depth_map - depth_map.min()) / (depth_map.max() - depth_map.min())

    return img, depth_map

# Test on an image
image_path = "room.jpg"  # Replace with your image
image, depth = estimate_depth(image_path)

# Display results
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.imshow(image)
plt.title("Original Image")

plt.subplot(1, 2, 2)
plt.imshow(depth, cmap="plasma")
plt.title("Depth Map")

plt.show()


error: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:199: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
